In [ ]:
import psycopg2
import numpy as np

# Connect to PostgreSQL (ensure pgvector is installed: CREATE EXTENSION IF NOT EXISTS vector;)
conn = psycopg2.connect(
    dbname="moviesdb",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# 1. CREATE TABLE
cur.execute("""
DROP TABLE IF EXISTS movies;
CREATE TABLE movies (
    id SERIAL PRIMARY KEY,
    title TEXT,
    year INT,
    rating FLOAT,
    embedding VECTOR(3)  -- set to your embedding dimension
);
""")
conn.commit()

# 2. INSERT MOVIES
movies = [
    ("The Matrix", 1999, 8.7, [0.1, 0.3, 0.5]),
    ("Inception", 2010, 8.8, [0.2, 0.1, 0.4]),
    ("Pulp Fiction", 1994, 8.9, [0.05, 0.2, 0.45]),
]
for m in movies:
    cur.execute("INSERT INTO movies (title, year, rating, embedding) VALUES (%s, %s, %s, %s);", m)
conn.commit()

# 3. QUERY (vector similarity + filter)
query_vec = np.array([0.1, 0.25, 0.45])
cur.execute("""
SELECT title, year, rating,
       embedding <-> %s::vector AS distance
FROM movies
WHERE year < 2000
ORDER BY embedding <-> %s::vector
LIMIT 5;
""", (query_vec.tolist(), query_vec.tolist()))

for row in cur.fetchall():
    print(f"{row[0]} ({row[1]}) | Rating: {row[2]} | Distance: {row[3]:.4f}")

cur.close()
conn.close()
